<a href="https://colab.research.google.com/github/ricotta-jpgomes/mod-multinivel-nuvem/blob/main/consolidacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configuração do ambiente

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
import sys

import pandas as pd
from sklearn.model_selection import train_test_split

from datetime import datetime, timedelta
import time
import requests

In [ ]:
! pip freeze > requirements.txt

In [ ]:
# Carregando variáveis de ambiente
load_dotenv('/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/.env')

True

In [ ]:
extraction_date = os.environ['EXTRACTION_DATE']

In [ ]:
aws_file = os.environ['AWS_FILE']
gcp_file = os.environ['GCP_FILE']
azure_file = os.environ['AZURE_FILE']

path = "/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Dados/"

In [ ]:
df_vms_gcp = pd.read_csv(f"{path}{gcp_file}")
df_vms_az = pd.read_csv(f"{path}{azure_file}")
df_vms_aws = pd.read_csv(f"{path}{aws_file}")

## Transformação

In [ ]:
df_vms = pd.concat([df_vms_aws, df_vms_gcp, df_vms_az], ignore_index=True, copy=False)
df_vms.info()

display(df_vms.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281375 entries, 0 to 281374
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   extractionDate    281375 non-null  object 
 1   provider          281375 non-null  object 
 2   machineType       281375 non-null  object 
 3   regionCode        281375 non-null  object 
 4   region            281375 non-null  object 
 5   numberOfCores     281375 non-null  int64  
 6   memoryInMiB       281375 non-null  int64  
 7   currencyCode      281375 non-null  object 
 8   priceOnDemand     281375 non-null  float64
 9   priceReserved1yr  94483 non-null   float64
 10  priceReserved3yr  94483 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 23.6+ MB


,extractionDate,provider,machineType,regionCode,region,numberOfCores,memoryInMiB,currencyCode,priceOnDemand,priceReserved1yr,priceReserved3yr
0,2025-07-06 22:37:59.288684-03:00,AWS,m6i.large,us-gov-west-1,AWS GovCloud (US-West),2,8192,USD,0.217,NaN,NaN
1,2025-07-06 22:37:59.288684-03:00,AWS,m7i-flex.12xlarge,us-gov-west-1,AWS GovCloud (US-West),48,196608,USD,2.981,NaN,NaN
2,2025-07-06 22:37:59.288684-03:00,AWS,m7g.large,ap-northeast-2,Asia Pacific (Seoul),2,8192,USD,0.110,NaN,NaN
3,2025-07-06 22:37:59.288684-03:00,AWS,m6idn.12xlarge,ap-southeast-1,Asia Pacific (Singapore),48,196608,USD,5.677,NaN,NaN
4,2025-07-06 22:37:59.288684-03:00,AWS,m6id.large,us-east-2,US East (Ohio),2,8192,USD,0.674,NaN,NaN


In [ ]:
df_vms.drop_duplicates(subset=['machineType', 'regionCode', 'numberOfCores', 'memoryInMiB', 'priceOnDemand'], inplace = True)
df_vms.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145300 entries, 0 to 281373
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   extractionDate    145300 non-null  object 
 1   provider          145300 non-null  object 
 2   machineType       145300 non-null  object 
 3   regionCode        145300 non-null  object 
 4   region            145300 non-null  object 
 5   numberOfCores     145300 non-null  int64  
 6   memoryInMiB       145300 non-null  int64  
 7   currencyCode      145300 non-null  object 
 8   priceOnDemand     145300 non-null  float64
 9   priceReserved1yr  46749 non-null   float64
 10  priceReserved3yr  46749 non-null   float64
dtypes: float64(3), int64(2), object(6)
memory usage: 13.3+ MB


In [ ]:
regions = df_vms['region'].unique().tolist()
print(regions)

['AWS GovCloud (US-West)', 'Asia Pacific (Seoul)', 'Asia Pacific (Singapore)', 'US East (Ohio)', 'US East (Miami)', 'Canada West (Calgary)', 'US East (N. Virginia)', 'Israel (Tel Aviv)', 'EU (Ireland)', 'Asia Pacific (Tokyo)', 'Asia Pacific (Mumbai)', 'South America (Sao Paulo)', 'EU (Frankfurt)', 'EU (Paris)', 'Asia Pacific (Hyderabad)', 'Argentina (Buenos Aires)', 'Mexico (Central)', 'EU (Stockholm)', 'US West (N. California)', 'Asia Pacific (Sydney)', 'Europe (Zurich)', 'EU (Milan)', 'Middle East (Bahrain)', 'Asia Pacific (Jakarta)', 'US East (Boston)', 'US West (Oregon)', 'Canada (Central)', 'Asia Pacific (Taipei)', 'US East (Chicago)', 'EU (London)', 'Peru (Lima)', 'AWS GovCloud (US-East)', 'Europe (Vodafone) - Berlin', 'Asia Pacific (Osaka)', 'Asia Pacific (Hong Kong)', 'Europe (Spain)', 'Middle East (UAE)', 'US West (Denver)', 'US East (New York City)', 'Nigeria (Lagos)', 'Asia Pacific (Melbourne)', 'US West (Los Angeles)', 'Asia Pacific (Malaysia)', 'Germany (Hamburg)', 'US Eas

In [ ]:
with open('regions.txt', 'w') as f:
    for region in regions:
        f.write(region + '\n')

In [ ]:
regions = {
    'Africa': [
        'Africa (Cape Town)',
        'Nigeria (Lagos)',
        'Morocco (Casablanca)',
        'Senegal (Dakar)',
        'Johannesburg, South Africa',
        'ZA North'
    ],
    'Asia': [
        'Asia Pacific (Hong Kong)',
        'Asia Pacific (Taipei)',
        'Asia Pacific (Tokyo)',
        'Taiwan (Taipei)',
        'Asia Pacific (KDDI) - Osaka',
        'Asia Pacific (KDDI) - Tokyo',
        'Asia Pacific (Seoul)',
        'Asia Pacific (SKT) - Daejeon',
        'Asia Pacific (SKT) - Seoul',
        'Asia Pacific (Osaka)',
        'Asia Pacific (Mumbai)',
        'India (Kolkata)',
        'India (Delhi)',
        'Asia Pacific (Hyderabad)',
        'Asia Pacific (Singapore)',
        'Thailand (Bangkok)',
        'Philippines (Manila)',
        'Asia Pacific (Jakarta)',
        'Asia Pacific (Malaysia)',
        'Asia Pacific (Thailand)',
        'Taiwan',
        'Singapore',
        'Tokyo, Japan',
        'Mumbai, India',
        'Hong Kong',
        'Osaka, Japan',
        'Seoul, South Korea',
        'Delhi, India',
        'Jakarta, Indonesia',
        'IN West',
        'SE Central',
        'KR South',
        'IN West Jio',
        'JA West',
        'JA East',
        'AP Southeast',
        'AP East',
        'IN South',
        'IN Central',
        'KR Central'
    ],
    'Oceania': [
        'Asia Pacific (Sydney)',
        'New Zealand (Auckland)',
        'Australia (Perth)',
        'Asia Pacific (Melbourne)',
        'Sydney, Australia',
        'Melbourne, Australia',
        'AU Central',
        'AU East',
        'AU Southeast'
    ],
    'Europe': [
        'EU (Frankfurt)',
        'Germany (Hamburg)',
        'Poland (Warsaw)',
        'Europe (Vodafone) - Berlin',
        'Europe (Vodafone) - Dortmund',
        'Europe (Vodafone) - Munich',
        'Europe (Zurich)',
        'EU (Stockholm)',
        'Denmark (Copenhagen)',
        'Finland (Helsinki)',
        'EU (Milan)',
        'Europe (Spain)',
        'EU (Ireland)',
        'EU (London)',
        'Europe (Vodafone) - London',
        'Europe (Vodafone) - Manchester',
        'Europe (British Telecom) - Manchester',
        'EU (Paris)',
        'Netherlands',
        'Belgium',
        'Frankfurt, Germany',
        'London, England, UK',
        'Warsaw, Poland',
        'Finland',
        'europe-north2',
        'europe-southwest1',
        'europe-west10',
        'europe-west12',
        'Zurich, Switzerland',
        'europe-west8',
        'europe-west9',
        'CH North',
        'DE West Central',
        'UK South',
        'NO East',
        'EU North',
        'EU West',
        'FR Central',
        'ES Central',
        'PL Central',
        'UK West',
        'IT North'
    ],
    'Middle East': [
        'Israel (Tel Aviv)',
        'Middle East (UAE)',
        'Middle East (Bahrain)',
        'Oman (Muscat)',
        'me-central1',
        'me-central2',
        'Tel Aviv, Israel',
        'AE North',
        'IL Central',
        'QA Central'
    ],
    'North America': [
        'Canada (Central)',
        'Canada (BELL) - Toronto',
        'Canada West (Calgary)',
        'Mexico (Central)',
        'US East (N. Virginia)',
        'US East (Atlanta)',
        'US East (Boston)',
        'US East (Chicago)',
        'US East (Dallas)',
        'US East (Houston)',
        'US East (Kansas City 2)',
        'US East (Miami)',
        'US East (Minneapolis)',
        'US East (New York City)',
        'US East (Philadelphia)',
        'Mexico (Queretaro)',
        'US East (Verizon) - Boston',
        'US East (Verizon) - Atlanta',
        'US East (Verizon) - Nashville',
        'US East (Verizon) - Chicago',
        'US East (Verizon) - Charlotte',
        'US East (Verizon) - Dallas',
        'US East (Verizon) - Detroit',
        'US East (Lenexa)',
        'US East (Verizon) - Houston',
        'US East (Verizon) - Miami',
        'US East (Verizon) - Minneapolis',
        'US East (Verizon) - New York',
        'US East (Verizon) - Tampa',
        'US East (Verizon) - Washington DC',
        'US East (Ohio)',
        'AWS GovCloud (US-East)',
        'AWS GovCloud (US-West)',
        'US West (N. California)',
        'US West (Oregon)',
        'US West (Denver)',
        'US West (Honolulu)',
        'US West (Las Vegas)',
        'US West (Los Angeles)',
        'US West (Portland)',
        'US West (Phoenix)',
        'US West (Seattle)',
        'US West (Verizon) - San Francisco Bay Area',
        'US West (Verizon) - Denver',
        'US West (Verizon) - Las Vegas',
        'US West (Verizon) - Los Angeles',
        'US West (Verizon) - Phoenix',
        'US West (Verizon) - Seattle',
        'Northern Virginia, USA',
        'Iowa, USA',
        'Oregon, USA',
        'Montréal, Canada',
        'Toronto, Canada',
        'northamerica-south1',
        'us-east5',
        'us-south1',
        'Los Angeles, USA',
        'Salt Lake City, USA',
        'Las Vegas, USA',
        'MX Central',
        'US East 2',
        'US West 2',
        'US East',
        'US West Central',
        'US South Central',
        'US West',
        'US Central',
        'CA Central',
        'CA East',
        'US North Central',
        'US West 3'
    ],
    'South America': [
        'South America (Sao Paulo)',
        'Argentina (Buenos Aires)',
        'Peru (Lima)',
        'Chile (Santiago)',
        'São Paulo, Brazil',
        'Santiago, Chile',
        'BR South'
    ]
}

In [ ]:
def normalize_regions(region):
    for continent, regions_list in regions.items():
        if region in regions_list:
            return continent
    return None

In [ ]:
df_vms['region'] = df_vms['region'].apply(normalize_regions)

### Amostragem

In [ ]:
# Calcula a contagem de registros para cada provedor
rows_per_provider = df_vms['provider'].value_counts()
print("Contagem original de registros por provedor:")
print(rows_per_provider)

# Encontra o provedor com o menor número de registros
min_rows_provider = rows_per_provider.idxmin()
print("\nProvedor com a menor quantidade de registros:")
print(min_rows_provider)

Contagem original de registros por provedor:
provider
AWS      103671
Azure     35194
GCP        6435
Name: count, dtype: int64

Provedor com a menor quantidade de registros:
GCP


In [ ]:
# Define o tamanho da amostra com base no provedor com o menor número de registros
sample = rows_per_provider.min()
print(f"\nO tamanho da amostra para cada provedor será: {sample}")


O tamanho da amostra para cada provedor será: 6435


In [ ]:
# Amostragem Estratificada por Grupo e Consolidação
samples_list = []

providers = df_vms['provider'].unique().tolist()
providers.remove(min_rows_provider)

print("\nExecutando a amostragem para cada provedor...\n")
for provider in providers:
    print(f"  -> Amostrando para o provedor: {provider}")

    # 1. Filtra o DataFrame para o provedor atual
    df_current_provider = df_vms[df_vms['provider'] == provider]

    # 2. Realiza a amostragem estratificada
    #    A estratificação é feita pela coluna 'regionCode'
    #    Usamos train_test_split para fazer isso de forma simples e robusta
    _, provider_sample = train_test_split(
        df_current_provider,
        test_size=sample,
        stratify=df_current_provider['region'], # Garante a proporção das regiões
        random_state=42 # Garante que a amostra seja a mesma toda vez que rodar
    )

    # 3. Adiciona a amostra gerada à nossa lista
    samples_list.append(provider_sample)

# 4. Concatena todas as amostras em um único DataFrame final
df_vms_gcp = df_vms[df_vms['provider'] == min_rows_provider]
samples_list.append(df_vms_gcp) # Inserindo o dataframe do provedor com menos registros

df_vms_final = pd.concat(samples_list, ignore_index=True)

print("\nProcesso de amostragem finalizado.")


Executando a amostragem para cada provedor...

  -> Amostrando para o provedor: AWS
  -> Amostrando para o provedor: Azure

Processo de amostragem finalizado.


In [ ]:

# Verificação do Resultado
print(f"Número total de registros no DataFrame final: {len(df_vms_final)}")
print("\nContagem de registros por provedor no DataFrame final:")
print(df_vms_final['provider'].value_counts())

display(df_vms_final.head())

Número total de registros no DataFrame final: 19305

Contagem de registros por provedor no DataFrame final:
provider
AWS      6435
Azure    6435
GCP      6435
Name: count, dtype: int64


,extractionDate,provider,machineType,regionCode,region,numberOfCores,memoryInMiB,currencyCode,priceOnDemand,priceReserved1yr,priceReserved3yr
0,2025-07-06 22:37:59.288684-03:00,AWS,m6idn.16xlarge,eu-west-1,Europe,64,262144,USD,7.464,NaN,NaN
1,2025-07-06 22:37:59.288684-03:00,AWS,m6id.xlarge,us-gov-west-1,North America,4,16384,USD,0.868,NaN,NaN
2,2025-07-06 22:37:59.288684-03:00,AWS,m5a.8xlarge,eu-west-2,Europe,32,131072,USD,1.946,1.354,1.037
3,2025-07-06 22:37:59.288684-03:00,AWS,m5ad.8xlarge,ca-central-1,North America,32,131072,USD,2.634,1.953,1.588
4,2025-07-06 22:37:59.288684-03:00,AWS,m7g.8xlarge,eu-west-3,Europe,32,131072,USD,1.619,1.100,0.790


In [ ]:
df_vms_final['region'].value_counts()

,count
region,
North America,6109
Europe,5432
Asia,4778
Oceania,1034
Middle East,992
South America,597
Africa,363


### Tratamento dos dados

In [ ]:
def obter_cotacao_dolar_bcb_para_data(data_alvo):
    """
    Busca a cotação de venda do dólar PTAX para uma data específica no Banco Central do Brasil.
    Se não houver cotação na data_alvo (fim de semana/feriado), tenta os 7 dias anteriores.
    Retorna a cotação como um float ou None se não encontrar.
    """
    if not isinstance(data_alvo, datetime):
        print("Aviso: data_alvo não é um objeto datetime. Tentando converter.")
        try:
            data_alvo = pd.to_datetime(data_alvo)
        except Exception as e:
            print(f"Não foi possível converter a data. Erro: {e}")
            return None

    print(f"Buscando cotação do dólar para a data de referência: {data_alvo.strftime('%d/%m/%Y')}")

    for i in range(8): # Tenta a data alvo e até 7 dias antes
        data_consulta = data_alvo - timedelta(days=i)
        data_formatada_api = data_consulta.strftime('%m-%d-%Y')

        url_bcb = (
            f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
            f"CotacaoDolarDia(dataCotacao=@dataCotacao)?@dataCotacao='{data_formatada_api}'"
            f"&$top=1&$format=json"
        )
        try:
            response = requests.get(url_bcb, timeout=10)
            response.raise_for_status()
            dados_api = response.json()

            if dados_api.get('value') and len(dados_api['value']) > 0:
                # Usamos a 'cotacaoVenda', que é a referência para o consumidor
                cotacao = dados_api['value'][0]['cotacaoVenda']
                print(f"-> Cotação encontrada para o dia {data_consulta.strftime('%d/%m/%Y')}: {cotacao}")
                return float(cotacao)

        except requests.exceptions.RequestException as e:
            print(f"--> Falha na consulta para o dia {data_consulta.strftime('%d/%m/%Y')}. Tentando dia anterior...")
            time.sleep(0.5) # Pequena pausa para não sobrecarregar

    print(f"Não foi possível obter cotação do BCB para {data_alvo.strftime('%d/%m/%Y')} ou 7 dias anteriores.")
    return None

In [ ]:
data_alvo = extraction_date
cotacao = obter_cotacao_dolar_bcb_para_data(data_alvo)
print(cotacao)

Aviso: data_alvo não é um objeto datetime. Tentando converter.
Buscando cotação do dólar para a data de referência: 07/06/2025
-> Cotação encontrada para o dia 06/06/2025: 5.5947
5.5947


In [ ]:
df_vms_final['priceOnDemand'] = round((df_vms_final['priceOnDemand'] * cotacao), 2)
df_vms_final['priceReserved1yr'] = round((df_vms_final['priceReserved1yr'] * cotacao), 2)
df_vms_final['priceReserved3yr'] = round((df_vms_final['priceReserved3yr'] * cotacao), 2)

In [ ]:
df_vms_final['currencyCode'] = 'BRL'

In [ ]:
df_vms_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19305 entries, 0 to 19304
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   extractionDate    19305 non-null  object 
 1   provider          19305 non-null  object 
 2   machineType       19305 non-null  object 
 3   regionCode        19305 non-null  object 
 4   region            19305 non-null  object 
 5   numberOfCores     19305 non-null  int64  
 6   memoryInMiB       19305 non-null  int64  
 7   currencyCode      19305 non-null  object 
 8   priceOnDemand     19305 non-null  float64
 9   priceReserved1yr  10065 non-null  float64
 10  priceReserved3yr  10065 non-null  float64
dtypes: float64(3), int64(2), object(6)
memory usage: 1.6+ MB


In [ ]:
df_vms_final.to_csv(f"{path}/vms_general.csv", index=False)